# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`MET-Data`*

---
*@author: Evan*\
*@date: 2023-08-09*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import sys
sys.path.append('../../src/')
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
chem = xr.open_dataset('D:/data/Project_Anqing/202307/COMBINE_ACONC_v54_intel_CN3AH_135X138_2023_chem.nc')
grid = xr.open_dataset('D:/data/Project_Anqing/GRIDCRO2D_2023141.nc')

In [3]:
# 将时间点转换为实际时间
times=pd.date_range(timestart,timeend,freq='h')
times

DatetimeIndex(['2023-07-01 00:00:00', '2023-07-01 01:00:00',
               '2023-07-01 02:00:00', '2023-07-01 03:00:00',
               '2023-07-01 04:00:00', '2023-07-01 05:00:00',
               '2023-07-01 06:00:00', '2023-07-01 07:00:00',
               '2023-07-01 08:00:00', '2023-07-01 09:00:00',
               ...
               '2023-07-31 14:00:00', '2023-07-31 15:00:00',
               '2023-07-31 16:00:00', '2023-07-31 17:00:00',
               '2023-07-31 18:00:00', '2023-07-31 19:00:00',
               '2023-07-31 20:00:00', '2023-07-31 21:00:00',
               '2023-07-31 22:00:00', '2023-07-31 23:00:00'],
              dtype='datetime64[ns]', length=744, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [5]:
# 结合地形高度HT与各气压层离地高度ZH获得各层实际海拔高度
ht=np.squeeze(grid.HT)
height=np.zeros([np.size(chem.ZH,0),
                 np.size(chem.ZH,1),
                 np.size(chem.ZH,2),
                 np.size(chem.ZH,3)])
for t in range(np.size(chem.ZH,0)):
    for l in range(np.size(chem.ZH,1)):
        height[t,l,:,:]=chem.ZH[t,l,:,:]+ht

In [6]:
days=1 # set spin-up days
dataset=xr.Dataset(
    data_vars=dict(
        # ! vars from CMAQ
        CO=(['time','level','y','x'],chem.CO[days*24-8:-8,:,:,:].data*30/22.4,{'long name':'Carbon Monoxide','units':'ug m-3'}),
        O3=(['time','level','y','x'],chem.O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Ozone','units':'ug m-3'}),
        NO=(['time','level','y','x'],chem.NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Nitric Oxide','units':'ug m-3'}),
        NO2=(['time','level','y','x'],chem.NO2[days*24-8:-8,:,:,:].data*37/22.4,{'long name':'Nitrogen Dioxide','units':'ug m-3'}),
        SO2=(['time','level','y','x'],chem.SO2[days*24-8:-8,:,:,:].data*64/22.4,{'long name':'Sulfur Dioxide','units':'ug m-3'}),
        VOC=(['time','level','y','x'],chem.VOC[days*24-8:-8,:,:,:].data,{'long name':'Volitile Organic Compounds','units':'ppbC'}),
        PM25=(['time','level','y','x'],chem.PM25_TOT[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        PM10=(['time','level','y','x'],chem.PM10[days*24-8:-8,:,:,:].data,{'long name':'','units':'ug m-3'}),
        # ! wwind
        wwind=(['time','level','y','x'],chem.WWind[days*24-8:-8,:,:,:].data,{'long name':'Vertical Wind Speed','units':'m s-1'}),
        # ! altitude
        HT=(['time','level','y','x'],height[days*24-8:-8,:,:,:].data,{'long name':'Altitudes','units':'m'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202307',
        grid='CN3AH_135X138',
        createtime=pd.Timestamp.now().strftime('%Y-%m-%d'),
    ),
)
dataset

<xarray.Dataset>
Dimensions:    (time: 744, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-07-01 ... 2023-07-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    CO         (time, level, y, x) float32 188.5 189.9 195.7 ... 38.77 38.74
    O3         (time, level, y, x) float32 97.16 114.4 ... 2.488e+03 2.49e+03
    NO         (time, level, y, x) float32 0.0003553 0.0003078 ... 2.27e-09
    NO2        (time, level, y, x) float32 2.661 2.087 1.574 ... 0.07547 0.07501
    SO2        (time, level, y, x) float32 0.7657 0.491 ... 0.009897 0.009882
    VOC        (time, level, y, x) float32 69.49 50.83 40.06 ... 0.04352 0.04321
    PM25       (time, level, y, x) float32 15.18 12.59 11.26 ... 0.0 0.0 0.0
    PM10       (time, level, y, x) float32 18.26 15.64 14.62 ... 0.0 0.0 0.0
    wwind      (time, level, y, x) float32 -0.05471 -0.1354 ... 0.001826
    HT         (time, level, y, x) float64 368.0 633.6 ... 1.98e+04 1.98e+04
Attributes:
    case:        Anqing_202307
    grid:        CN3AH_135X138
    createtime:  2023-08-09

In [7]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset.data_vars}
dataset.to_netcdf('D:/Download/AQ_2307_chem.nc',encoding=encoding)